# Aldy: Data Setup
---

This notebook documets all of the commands used to perform experiments and generate data from the Aldy paper.

## Initial steps

First, we will fetch the software needed for our experiments, and also define the gene regions of interest:

In [2]:
%%bash
cat <<- EOF > regions.txt
1:97541298-98388615     # DPYD
6:18126541-18157374     # TPMT
6:31970000-32010000     # CYP21A
7:99245000-99278000     # CYP3A5
7:99354000-99465000     # CYP3A4
10:96445000-96615000    # CYP2C19
10:96691000-96754000    # CYP2C9
10:96796000-96830000    # CYP2C8
10:135337000-135352620  # CYP2E1
12:21282127-21394730    # SLCO1B1
15:75010000-75017877    # CYP1A1
15:75038183-75048941    # CYP1A2
16:28488000-28504000    # CLN3
19:15619000-16009500    # CYP4F2
19:41347500-41400000    # CYP2A6
19:41428000-41525000    # CYP2B6
22:42518900-42553000    # CYP2D6
EOF

git clone https://github.com/sfu-compbio/cypiripi ~/cypiripi
cd ~/cypiripi
git checkout 44b8949 
cd -
git clone https://github.com/sfu-compbio/mrsfast ~/mrsfast
cd ~/mrsfast
git checkout 32dda6a
make -j 
cd -
git clone https://github.com/BilkentCompGen/mrfast ~/mrfast
cd ~/mrfast
git checkout e8f70f7
make -j 
cd - 

# manually wget astrolabe-0.7.5.tar.gz from 
# https://www.childrensmercy.org/Health_Care_Professionals/Research/Pediatric_Genomic_Medicine/Software_Tools/
# because of EULA 

git clone git@bitbucket.org:compbio/aldy.git ~/aldy
cd ~/aldy
git checkout 832281c 
cd -
export PATH=$PATH:~/mrsfast:~/mrfast:~/aldy/python:~/cypiripi/distribution

Samtools with HTTPS support is necessary for BAM retrieval. It can be compiled with:

```./configure --enable-configure-htslib --enable-libcurl 
   (--disable-bz2 --disable-lzma)```

## Illumina Data

In [1]:
%%bash
mkdir illumina-1000genomes
cd illumina-1000genomes
mkdir -p vcfs bams cypiripi aldy astrolabe

#### Fetch BAM files

In [ ]:
%%bash
function get_bam() {
    sample="$1"
    bam_url="$2"
    prefix="$3"
    samtools view -H $bam_url > bams/${sample}.sam
    for region in `cut -d' ' -f1 ../regions.txt`; do
        samtools view $bam_url ${prefix}$region >> bams/${sample}.sam
    done
    samtools view -bS bams/${sample}.sam -o bams/${sample}.bam
    samtools index bams/${sample}.bam
    rm -rf bams/${sample}.sam
}
get_bam NA12877 https://storage.googleapis.com/genomics-public-data/platinum-genomes/bam/NA12877_S1.bam chr
get_bam NA12878 https://storage.googleapis.com/genomics-public-data/platinum-genomes/bam/NA12878_S1.bam chr
get_bam NA12889 https://storage.googleapis.com/genomics-public-data/platinum-genomes/bam/NA12889_S1.bam chr
get_bam NA12890 https://storage.googleapis.com/genomics-public-data/platinum-genomes/bam/NA12890_S1.bam chr
get_bam NA12891 https://storage.googleapis.com/genomics-public-data/platinum-genomes/bam/NA12891_S1.bam chr
get_bam NA12892 https://storage.googleapis.com/genomics-public-data/platinum-genomes/bam/NA12892_S1.bam chr
get_bam NA11832 https://storage.googleapis.com/genomics-public-data/ftp-trace.ncbi.nih.gov/1000genomes/ftp/phase3/data/NA11832/alignment/NA11832.mapped.ILLUMINA.bwa.CEU.low_coverage.20120522.bam
get_bam NA19238 https://storage.googleapis.com/genomics-public-data/ftp-trace.ncbi.nih.gov/1000genomes/ftp/phase3/data/NA19238/high_coverage_alignment/NA19238.mapped.ILLUMINA.bwa.YRI.high_coverage_pcr_free.20130924.bam
get_bam NA19239 https://storage.googleapis.com/genomics-public-data/ftp-trace.ncbi.nih.gov/1000genomes/ftp/phase3/data/NA19239/high_coverage_alignment/NA19239.mapped.ILLUMINA.bwa.YRI.high_coverage_pcr_free.20130924.bam
get_bam NA19240 https://storage.googleapis.com/genomics-public-data/ftp-trace.ncbi.nih.gov/1000genomes/ftp/phase3/data/NA19240/high_coverage_alignment/NA19240.mapped.ILLUMINA.bwa.YRI.high_coverage_pcr_free.20130924.bam
get_bam NA19900 https://storage.googleapis.com/genomics-public-data/ftp-trace.ncbi.nih.gov/1000genomes/ftp/phase3/data/NA19900/alignment/NA19900.mapped.ILLUMINA.bwa.ASW.low_coverage.20120522.bam
get_bam NA10860 https://export.uppmax.uu.se/a2009002/opendata/HiSeqX_CEPH/NA10860-PCR-free-150pM/03-BAM/NA10860-PCR-free-150pM.clean.dedup.recal.bam
get_bam NA11992 https://export.uppmax.uu.se/a2009002/opendata/HiSeqX_CEPH/CEP-NA11992/03-BAM/CEP-NA11992.clean.dedup.recal.bam
get_bam NA11993 https://export.uppmax.uu.se/a2009002/opendata/HiSeqX_CEPH/NA11993-PCR-free-150pM/03-BAM/NA11993-PCR-free-150pM.clean.dedup.recal.bam

# The following samples were obtained from dbGaP:
#   NA12879, NA12880, NA12881, NA12882, NA12883, NA12884, 
#   NA12885, NA12886, NA12887, NA12888, NA12893

#### Fetch VCF files

In [ ]:
%%bash
function get_1000genomes_vcf() {
    sample="$1"
    for region in `cut -d' ' -f1 ../regions.txt`; do
        if [ "$sample" == "NA19240" ] ; then
            url="ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/supporting/related_samples_vcf/ALL.chr${region/\:*/}.phase3_shapeit2_mvncall_integrated_v5_related_samples.20130502.genotypes.vcf.gz"
        else
            url="ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr${region/\:*/}.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz"
        fi
        echo "$sample $url"
        tabix -h $url $region | vcf-subset -c $sample > ${region}.tmp
    done
    vcf-concat *.tmp | vcf-sort > "vcfs/${sample}.vcf"
}
get_1000genomes_vcf NA11832
get_1000genomes_vcf NA19238
get_1000genomes_vcf NA19239
get_1000genomes_vcf NA19900
get_1000genomes_vcf NA19240 

function get_platinum_vcf() {
    sample="$1"
    for region in `cut -d' ' -f1 ../regions.txt`; do
        tabix -h vcfs/High_Confidence_Calls_HG19.vcf.gz "chr$region" | vcf-subset -c $sample > ${region}.tmp
    done
    vcf-concat *.tmp | vcf-sort > "vcfs/${sample}_S1.genome.vcf"
}
get_platinum_vcf NA12879
get_platinum_vcf NA12880
get_platinum_vcf NA12881
get_platinum_vcf NA12882
get_platinum_vcf NA12883
get_platinum_vcf NA12884
get_platinum_vcf NA12885
get_platinum_vcf NA12886
get_platinum_vcf NA12887
get_platinum_vcf NA12888
get_platinum_vcf NA12893

wget -c -P vcfs https://storage.googleapis.com/genomics-public-data/platinum-genomes/vcf/NA12877_S1.genome.vcf
wget -c -P vcfs https://storage.googleapis.com/genomics-public-data/platinum-genomes/vcf/NA12878_S1.genome.vcf
wget -c -P vcfs https://storage.googleapis.com/genomics-public-data/platinum-genomes/vcf/NA12889_S1.genome.vcf
wget -c -P vcfs https://storage.googleapis.com/genomics-public-data/platinum-genomes/vcf/NA12890_S1.genome.vcf
wget -c -P vcfs https://storage.googleapis.com/genomics-public-data/platinum-genomes/vcf/NA12891_S1.genome.vcf
wget -c -P vcfs https://storage.googleapis.com/genomics-public-data/platinum-genomes/vcf/NA12892_S1.genome.vcf
wget -c -P vcfs https://export.uppmax.uu.se/a2009002/opendata/HiSeqX_CEPH/NA10860-PCR-free-150pM/04-VCF/NA10860-PCR-free-150pM.clean.dedup.recal.bam.recalibrated.indel.annotated.vcf
wget -c -P vcfs https://export.uppmax.uu.se/a2009002/opendata/HiSeqX_CEPH/NA10860-PCR-free-150pM/04-VCF/NA10860-PCR-free-150pM.clean.dedup.recal.bam.recalibrated.snp.annotated.vcf
wget -c -P vcfs https://export.uppmax.uu.se/a2009002/opendata/HiSeqX_CEPH/NA11993-PCR-free-150pM/04-VCF/NA11993-PCR-free-150pM.clean.dedup.recal.bam.recalibrated.indel.annotated.vcf
wget -c -P vcfs https://export.uppmax.uu.se/a2009002/opendata/HiSeqX_CEPH/NA11993-PCR-free-150pM/04-VCF/NA11993-PCR-free-150pM.clean.dedup.recal.bam.recalibrated.snp.annotated.vcf
wget -c -P vcfs https://export.uppmax.uu.se/a2009002/opendata/HiSeqX_CEPH/CEP-NA11992/04-VCF/CEP-NA11992.clean.dedup.recal.bam.recalibrated.indel.annotated.vcf
wget -c -P vcfs https://export.uppmax.uu.se/a2009002/opendata/HiSeqX_CEPH/CEP-NA11992/04-VCF/CEP-NA11992.clean.dedup.recal.bam.recalibrated.snp.annotated.vcf
    
cd vcfs
for i in NA128*.vcf ; do
    perl -pe 's/^chr//' $i > ${i/_S1.genome/}
done

parallel -j15 --eta bgzip -f {} ::: *.vcf
vcf-concat CEP-NA11992.clean.dedup.recal.bam.recalibrated.indel.annotated.vcf.gz CEP-NA11992.clean.dedup.recal.bam.recalibrated.snp.annotated.vcf.gz  | \ 
    vcf-sort | bgzip -c > NA11992.vcf.gz
vcf-concat NA10860-PCR-free-150pM.clean.dedup.recal.bam.recalibrated.indel.annotated.vcf.gz NA10860-PCR-free-150pM.clean.dedup.recal.bam.recalibrated.snp.annotated.vcf.gz | \
    vcf-sort | bgzip -c > NA10860.vcf.gz
vcf-concat NA11993-PCR-free-150pM.clean.dedup.recal.bam.recalibrated.indel.annotated.vcf.gz NA11993-PCR-free-150pM.clean.dedup.recal.bam.recalibrated.snp.annotated.vcf.gz | \
    vcf-sort | bgzip -c > NA11993.vcf.gz        

rm -rf *.annotated.vcf.gz *_S1.genome.vcf.gz
parallel -j15 --eta tabix -p vcf {} ::: *.vcf.gz

cd ..
rm -rf *.tmp *.tbi

#### Run Astrolabe

Astrolabe only supports GRCh37.

In [ ]:
%%bash
for i in NA12877 NA12878 NA12889 NA12890 NA12891 NA12892 NA12879 NA12880 NA12881 NA12882 NA12883 NA12884 NA12885 NA12886 NA12887 NA12888 NA12893 ; do 
    samtools view -H bams/${i}.bam |  sed  -e 's/SN:chr\([0-9XY]*\)/SN:\1/' | \
        samtools reheader - bams/${i}.bam > "bams/${i}.NCBI.bam" 
    samtools index "bams/${i}.NCBI.bam" 
    ~/astrolabe-0.7.5/run-astrolabe.sh \
        -inputVCF vcfs/${i}.vcf.gz \
        -inputBam bams/${i}.NCBI.bam \
        -conf ~/astrolabe-0.7.5/astrolabe.ini \
        -outFile astrolabe/${i}.log 
done

for i in NA10860 NA11832 NA11992 NA11993 NA19900 ; do 
    ~/astrolabe-0.7.5/run-astrolabe.sh \
        -inputVCF vcfs/${i}.vcf.gz \
        -inputBam bams/${i}.bam \
        -conf ~/astrolabe-0.7.5/astrolabe.ini \
        -outFile astrolabe/${i}.log  
done

# VCFs (extracted from 1000 genomes) here do not match BAMs 
# (which are sequenced afterwards with hi-coverage), 
# so ignore BAMs
for i in NA19238 NA19239 NA19240 ; do 
    ~/astrolabe-0.7.5/run-astrolabe.sh \
        -inputVCF vcfs/${i}.vcf.gz \
        -conf ~/astrolabe-0.7.5/astrolabe.ini \
        -outFile astrolabe/${i}.log  
done

#### Run Cypiripi

Platinum Genomes (NA128*) FASTQs, as well as NA19900 from 1000 genomes, are extracted from BAM files. Platinum Genomes are sequenced with 2x101bp 50x coverage.

In [ ]:
for i in bams/NA128*.bam bams/NA19900.bam; do
    sample=${i:5:7}
    samtools sort -n bams/${sample}.bam > ${sample}.qsort 
    bedtools bamtofastq -i ${sample}.qsort -fq f1.fq -fq2 f2.fq >/dev/null 2>&1 
    paste f1.fq f2.fq  | paste - - - - | awk -v OFS="\n" -v FS="\t" '{print($1,$3,$5,$7,$2,$4,$6,$8)}' |\
        sed 's/\/[12]$//' > cypiripi/${sample}.fq
    rm -rf ${sample}.qsort f1.fq f2.fq
    echo " ${sample} done! "
done

parallel --eta \
    'cypiripi.py -r ~/cypiripi/distribution/reference -f {} >{}.maplog 2>&1' ::: NA1{28,99}*.fq
parallel --eta \
    'cypiripi -f ~/cypiripi/distribution/reference.combined.align \
       -s {} -C 25 -E 500 -T 7 >{}.results 2>&1' ::: NA128*.sam
                
# IMPORTANT NOTE: cypiripi-fixed changes:
# ilp.cc:105@unwindMultiSNP must be commented out (or McAssert should be changed to E) for this to run
parallel --eta \
    'cypiripi-fixed -f ~/cypiripi/distribution/reference.combined.align \
       -s {} -C 25 -E 500 -T 7 >{}.results 2>&1' ::: NA128{77,90,92}*.sam
parallel --eta \
    'cypiripi-fixed -f ~/cypiripi/distribution/reference.combined.align \
       -s {} -C 20 -E 500 -T 5 >{}.results 2>&1' ::: NA128{84,89,79,80}*.sam
cypiripi-fixed -f ~/cypiripi/distribution/reference.combined.align \
    -s NA19900.fq.sam -C 7 -E 500 -T 2 >NA19900.fq.sam.results 2>&1

1000 Genome samples NA11832 is not compatible because the read lengths are variable.
Sample NA11992 has corrupted FASTQs, so data for Cypiripi was not generated.
Other samples have clipped BAMs, so we need to download FASTQ files:

In [ ]:
%%bash
mkdir tmp
cd tmp
wget -c ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR309/ERR309932/ERR309932_1.fastq.gz
wget -c ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR309/ERR309932/ERR309932_2.fastq.gz
wget -c ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR309/ERR309933/ERR309933_1.fastq.gz
wget -c ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR309/ERR309933/ERR309933_2.fastq.gz
wget -c ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR309/ERR309934/ERR309934_1.fastq.gz
wget -c ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR309/ERR309934/ERR309934_2.fastq.gz
wget -c https://export.uppmax.uu.se/a2009002/opendata/HiSeqX_CEPH/NA10860-PCR-free-150pM/02-FASTQ/NA10860-PCR-free-150pM_S1_L007_R1_001.fastq.gz
wget -c https://export.uppmax.uu.se/a2009002/opendata/HiSeqX_CEPH/NA10860-PCR-free-150pM/02-FASTQ/NA10860-PCR-free-150pM_S1_L007_R2_001.fastq.gz
wget -c https://export.uppmax.uu.se/a2009002/opendata/HiSeqX_CEPH/NA11993-PCR-free-150pM/02-FASTQ/NA11993-PCR-free-150pM_S1_L004_R1_001.fastq.gz
wget -c https://export.uppmax.uu.se/a2009002/opendata/HiSeqX_CEPH/NA11993-PCR-free-150pM/02-FASTQ/NA11993-PCR-free-150pM_S1_L004_R2_001.fastq.gz
parallel 'pigz -d -c {} | split -l 16000000 - split/{}' ::: *.fastq.gz

mkdir cypiripi
for i in ERR*_1.fastq.gz?? ; do
    paste $i ${i/_1/_2} | paste - - - - | \
        awk -v OFS="\n" -v FS="\t" '{print($1,$3,$5,$7,$2,$4,$6,$8)}' | sed 's/\/[12]$//' \
        > cypiripi/${i/_1/}.fq
done
for i in NA*_R1*.fastq.gz?? ; do
    paste $i ${i/R1/R2} | paste - - - - | \
        awk -v OFS="\n" -v FS="\t" '{print($1,$3,$5,$7,$2,$4,$6,$8)}' | sed 's/\/[12]$//' \
        > cypiripi/${i/R1/}.fq
done

function align() {
    i="$1"
    mrserr="$2"
    mrerr="$3"
    pemin="$4"
    pemax="$5"
    ~/mrsfast/mrsfast --search ~/cypiripi/distribution/reference.cyp2d6.fa -e $mrserr --seqcomp \
         --seq ${i} -o ${i}.cyp2D6.sam --disable-sam-header --disable-nohits
    ~/mrfast/mrfast --search ~/cypiripi/distribution/reference.cyp2d7.fa -e $mrerr --seqcomp \
        --seq ${i} -o ${i}.cyp2D7.sam -u ${i}.X1.sam.unmap
    rm -rf ${i}.X1.sam.unmap
    ~/mrsfast/mrsfast --search ~/cypiripi/distribution/reference.cyp2d6.fa -e $mrserr --seqcomp \
        --pe --seq ${i} -o ${i}.cyp2D6.sam.paired --disable-sam-header --disable-nohits --min $pemin --max $pemax
    ~/mrfast/mrfast --search ~/cypiripi/distribution/reference.cyp2d7.fa -e $mrerr --seqcomp \
        --pe --seq ${i} -o ${i}.cyp2D7.sam.paired -u ${i}.X2.sam.unmap --min $pemin --max $pemax
    rm -rf ${i}.X2.sam.unmap
    
    # On SGE cluster, this qsub runs this function without any problems:
    # parallel -j1 qsub -cwd -V -b y -N "map{}" -l h_vmem=14G -l h_rt=8:00:00 -l h_stack=8 ...
}

cd cypiripi
parallel align {} 3  8 150  800 ::: NA*PCR-free*.fq # These samples are 2x150bp 30x
parallel align {} 5 12 200 1000 ::: ERR*.fq # These samples are roughly 2x250bp 25x

for i in ERR309932 ERR309933 ERR309934 NA11993 NA10860 ; do
    cat ${i}*.cyp2D6.sam ${i}*.cyp2D7.sam > ${i}.sam
    cat ${i}*.cyp2D6.sam.paired ${i}*.cyp2D7.sam.paired > ${i}.sam.paired
    cypiripi.py -r ~/cypiripi/distribution/reference -s ${i}.sam
done
zmv -W 'ERR309932.sam*' 'NA19238.sam*'
zmv -W 'ERR309933.sam*' 'NA19239.sam*'
zmv -W 'ERR309934.sam*' 'NA19240.sam*'
mv NA*.sam* ../../

cd ../..
rm -rf tmp

parallel 'cypiripi -f ~/cypiripi/distribution/reference.combined.align \
    -s {} -C 14 -E 500 -T 3 >{}.results 2>&1' ::: NA192*.sam
parallel 'cypiripi -f ~/cypiripi/distribution/reference.combined.align \
    -s {} -C 13 -E 500 -T 3 >{}.results 2>&1' ::: NA1{08,19}*.sam

#### Run Aldy

In [ ]:
%%bash
parallel --eta aldy.py -p illumina {} -o aldy/{/}.log ::: bams/*.bam

## PGRNseq data (BCM-HGSC)

In [ ]:
%%bash
mkdir pgrnseq-baylor
cd pgrnseq-baylor
mkdir -p vcfs cypiripi aldy astrolabe

JAVA="../software/jre-1.7.0_80/bin/java -Xmx16g -jar"
PICARD="../software/picard-1.140/picard.jar"
BWA="../software/bwa-0.7.12/bwa"
GATK="../software/gatk-2.5-2/GenomeAnalysisTK-2.5-2.jar"
GATK3="../software/gatk-3.3-0/GenomeAnalysisTK-3.3-0.jar"

ref="../resources/human_g1k_v37.fasta"
known_snp="../resources/dbsnp_138.b37.vcf.gz"
known_indel="../resources/1000G_phase1.indels.b37.vcf.gz"
known_indel_2="../resources/Mills_and_1000G_gold_standard.indels.b37.vcf.gz"

Obtained from BCM-HGSC:

- `amplicons_fastq/`
- `bams/` (already aligned according to the procedure below)
- `fastq/`

#### Alignment

In [ ]:
%%bash
sample=$1
outdir="gatk-calls/${sample}"
mkdir -p ${outdir} ${outdir}/tmp

function align() { # Alignment (Baylor samples are already aligned)
    $BWA aln $ref ${sample}.fq.1 -t 4 > ${outdir}/${sample}.sai.1
    $BWA aln $ref ${sample}.fq.2 -t 4 > ${outdir}/${sample}.sai.2
    $BWA sampe $ref ${outdir}/${sample}.sai.1 ${outdir}/${sample}.sai.2 ${sample}.fq.1 ${sample}.fq.2 \
        -r "@RG\tID:${sample}\tSM:${sample}\tPL:illumina\tLB:${sample}" | \
        samtools view -Sbh - -o ${outdir}/${sample}.hg19.bam
    $JAVA $PICARD SortSam VALIDATION_STRINGENCY=SILENT SO=coordinate \
        I=${outdir}/${sample}.hg19.bam O=${outdir}/${sample}.hg19.sort.bam \
        TMP_DIR=./${outdir}/tmp/
    $JAVA $PICARD MarkDuplicates VALIDATION_STRINGENCY=SILENT AS=true \
      I=${outdir}/${sample}.hg19.sort.bam O=${outdir}/${sample}.hg19.marked.bam \
      M=${outdir}/${sample}.hg19.marked.metrics TMP_DIR=./${outdir}/tmp/
    samtools index ${outdir}/${sample}.hg19.marked.bam
}
function realign() { # Realignment (Baylor samples are already realigned)
    $JAVA $GATK -T RealignerTargetCreator -nt 4 \
        -I ${outdir}/${sample}.hg19.marked.bam -R $ref \
        -o ${outdir}/${sample}.hg19.GATKrealign.intervals \
        -known $known_indel -known $known_snp -known $known_indel_2 \
        --downsampling_type NONE
    $JAVA $GATK -T IndelRealigner \
        -I ${outdir}/${sample}.hg19.marked.bam -R $ref \
        -o ${outdir}/${sample}.hg19.realigned.bam \
        -targetIntervals ${outdir}/${sample}.hg19.GATKrealign.intervals \
        -known $known_snp -known $known_indel -known $known_indel_2 \
        --downsampling_type NONE --consensusDeterminationModel USE_READS
    $JAVA $GATK -T BaseRecalibrator -nct 4 \
        -I ${outdir}/${sample}.hg19.realigned.bam -R $ref \
        -o ${outdir}/${sample}.hg19.GATKrecal_data.grp \
        -knownSites $known_snp -knownSites $known_indel -knownSites $known_indel_2 \
        --downsampling_type NONE -cov ReadGroupCovariate \
        -cov QualityScoreCovariate -cov CycleCovariate -cov ContextCovariate \
    $JAVA -Djava.io.tmpdir=./${sample}/tmp/ -jar $GATK -T PrintReads -nct 4 \
        -I ${sample}/${sample}.hg19.realigned.bam -R $ref \
        -o ${sample}/${sample}.hg19.realigned.recal.bam \
        -BQSR ${sample}/${sample}.hg19.GATKrecal_data.grp \
        --emit_original_quals --downsampling_type NONE 
}

#### VCF generation

In [ ]:
%%bash
function getvcf() { # VCF calling
    bam="$1" # ${sample}/${sample}.hg19.realigned.recal.bam
    extra="$2"
    $JAVA $GATK3 -T HaplotypeCaller -nct 8 \
        -I ${bam} -R $ref \
        -o ${outdir}/${sample}.hg19.raw.snps.indels.g.vcf
        --dbsnp $known_snp \
        --emitRefConfidence GVCF --variant_index_type LINEAR --variant_index_parameter 128000
    $JAVA $GATK3 -T GenotypeGVCFs \
        --variant ${outdir}/${sample}.hg19.raw.snps.indels.g.vcf -R $ref \
        -o ${outdir}/${sample}.hg19.raw.snps.indels.vcf
    $JAVA $GATK3 -T VariantAnnotator \
        -I ${bam} -R $ref \
        -o ${outdir}/${sample}.hg19.raw.snps.indels.annotate.vcf \
        --dbsnp $known_snp --variant ${outdir}/${sample}.hg19.raw.snps.indels.vcf \
        -L ${outdir}/${sample}.hg19.raw.snps.indels.vcf \
        -A Coverage -A BaseQualityRankSumTest -A FisherStrand -A HaplotypeScore \
        -A InbreedingCoeff -A MappingQualityRankSumTest
    $JAVA $GATK3 -T VariantRecalibrator \
        -input ${outdir}/${sample}.hg19.raw.snps.indels.annotate.vcf -R $ref \
        -recalFile ${outdir}/${sample}.hg19.raw.snps.recal \
        -tranchesFile ${outdir}/${sample}.hg19.raw.snps.tranches \
        -rscriptFile ${outdir}/${sample}.hg19.recal.plots.R \
        -resource:dbsnp,VCF,known=false,training=true,truth=true,prior=6.0 $known_snp \
        -an BaseQRankSum -an DP -an FS -an HaplotypeScore -an MQ -an MQRankSum -an QD -an ReadPosRankSum \
        -mode BOTH $extra
    $JAVA $GATK3 -T ApplyRecalibration \
        -input ${outdir}/${sample}.hg19.raw.snps.indels.annotate.vcf -R $ref \
        -o ${outdir}/${sample}.hg19.SNPs.vcf \
        -recalFile ${outdir}/${sample}.hg19.raw.snps.recal \
        -tranchesFile ${outdir}/${sample}.hg19.raw.snps.tranches \
        -mode BOTH -ts_filter_level 99.0 
}

Samples NA07345, NA19129, NA19202 and NA19770 need `--maxGaussians 4` for VariantRecalibrator (otherwise, VariantRecalibrator cannot find any data). Same goes for PGXT107, PGXT120, PGXT125, PGXT135, PGXT147, PGXT178, PGXT179, PGXT180 and PGXT223 in CDC data.

In [ ]:
%%bash
for i in bams/*.bam ; do getvcf $i ; done
for i in NA07345 NA19129 NA19202 NA19770 ; do getvcf "bams/${i}.bam" '--maxGaussians 4'; done
rsync -Pvza 'gatk-calls/*/*.SNPs.vcf' vcfs/
rm -rf gatk-calls
cd vcfs
zmv -W '*.hg19.SNPs.vcf' '*.vcf'
parallel -j15 --eta bgzip -f {} ::: *.vcf
parallel -j15 --eta tabix -p vcf {} ::: *.vcf.gz

#### Run Astrolabe


In [ ]:
%%bash
for i in bams/*.bam ; do 
    j=${i:5:7};
    ~/astrolabe-0.7.5/run-astrolabe.sh \
        -inputVCF vcfs/${j}.vcf.gz -inputBam $i \
        -outFile astrolabe/${j}.log \
        -conf ~/astrolabe-0.7.5/astrolabe.ini
done

#### Run Cypiripi

In [ ]:
%%bash
for i in fastq/*.fq.1.bz2 ; do
    sample=${i:6:7}
    paste <(bzip2 -d -c $i) <(bzip2 -d -c ${i/.1/.2}) | paste - - - - | \
        awk -v OFS="\n" -v FS="\t" '{print($1,$3,$5,$7,$2,$4,$6,$8)}' > cypiripi/${sample}.i.fq
done

cd cypiripi
parallel 'cypiripi.py -r ~/cypiripi/distribution/reference -f {} >{}.execlog 2>&1' ::: *.i.fq

parallel --eta 'cypiripi -f ~/cypiripi/distribution/reference.combined.align \
    -s {} -C 350 -E 10000 -T 87 >{}.results 2>&1' ::: *.sam
parallel --eta 'cypiripi -f ~/cypiripi/distribution/reference.combined.align \
    -s {} -C 330 -E 10000 -T 82 >{}.results 2>&1' ::: NA19701*.sam
parallel --eta 'cypiripi -f ~/cypiripi/distribution/reference.combined.align \
    -s {} -C 420 -E 10000 -T 105 >{}.results 2>&1' ::: \
    {HG00423,HG00464,HG01061,HG01062,HG01981,NA10861,NA12400,NA12891,NA18507,NA19818,NA19834,NA19902}*.sam
parallel --eta 'cypiripi -f ~/cypiripi/distribution/reference.combined.align \
    -s {} -C 600 -E 10000 -T 150 -F >log_350/{}.results 2>&1' ::: \
    {HG00465,HG00463}*.sam

## HG01981 / NA19127 WAT?
# NA19701 better coverage! // 330!

### -C 450 -E 10000 -T 112 ::: NA18517 

#### Run Aldy

In [ ]:
%%bash

for g in CYP2D6 CYP2A6 CYP2C19 CYP2C8 CYP2C9 CYP3A4 CYP3A5 CYP4F2 TPMT DPYD; do
    parallel --eta aldy.py -p pgrnseq-v2 {} -g $g -o aldy/{/}-${g}.log ::: bams/*.bam;
done

## PGXT (CDC) samples

In [ ]:
%%bash
cd vcfs
for i in {101..193}; do 
    time vcf-concat SNP/PGXT${i}.SNPs.vcf INDEL/PGXT${i}.INDELs.vcf | vcf-sort > PGXT${i}.vcf  
done
for i in {193..241}; do 
    time vcf-concat SNP/PGXT${i}.SNPs_Annotated.vcf INDEL/PGXT${i}.INDELs_Annotated.vcf | vcf-sort > PGXT${i}.vcf  
done

parallel -j15 --eta bgzip -f {} ::: *.vcf
parallel -j15 --eta tabix -p vcf {} ::: *.vcf.gz

cd ..
mkdir aldy
for g in CYP2D6 CYP2A6 CYP2C19 CYP2C8 CYP2C9 CYP3A4 CYP3A5 CYP4F2 TPMT DPYD; do
    echo $g;
    parallel --eta aldy.py -p pgrnseq-v1 {} -g $g -o aldy/{/}-${g}.log ::: bams/*.bam;
done